In [1]:
# WIDESCREEN HD BRO

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
#---------------------------------------------------------------------------------------------------------------------
#           CODED BY MICHAEL KEEN (CONTRACTOR)
#           COMCAST SENIOR DATA QUALITY ASSURANCE ANALYST 
#           5/2/2019
#---------------------------------------------------------------------------------------------------------------------

In [4]:
#---------------------------------------------------------------------------------------------------------------------
#           IMPORT VARIOUS LIBRARIES NEEDED FOR PROGRAM
#---------------------------------------------------------------------------------------------------------------------  
try:
    import json
except ImportError:
    import simplejson as json
    
import time

import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.animation as animation
plt.rcParams['figure.figsize'] = 12,5
%matplotlib qt
style.use('fivethirtyeight')

import csv
import unicodecsv
import pandas as pd 
import numpy as np
  
from textblob import TextBlob 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import emoji
import re
import tweepy

from http.client import IncompleteRead as http_incompleteRead
from urllib3.exceptions import IncompleteRead as urllib3_incompleteRead
from urllib3.exceptions import ProtocolError 
import urllib.parse

from IPython.display import clear_output

start = time.time()
freqvar = '1Min'
coiner = 0

In [5]:
#---------------------------------------------------------------------------------------------------------------------
#           CREATE (OR OVERWRITE IF EXISTING AND START FROM SCRATCH IN) SAVE FILES FOR TWEETS AND RETWEETS
#---------------------------------------------------------------------------------------------------------------------  
# with open('tweets.csv', 'wb') as file:
#     writer = unicodecsv.writer(file, delimiter = ',', quotechar = '"')
#     # Write header row.
#     writer.writerow(["TWEET", "USER_ID", "TWEET_ID", "TWT_POST_DT", "USER_NM", "USER_ACCT", "FOLLOWER_CNT", "FRIENDS_CNT",
#                     "LOCATION", "ACCT_CREATE_DT", "GEO_ENABLED", "USER_URL", "RETWEET_CNT", "COMPSENTIMENT", "POSSENTIMENT",
#                     "NEGSENTIMENT", "NEUSENTIMENT", "SENTIMENT_OA", "POSCNT", "NEGCNT", "BLOBPOL", "BLOBSUB", "COORDINATES", "VERIFIED_USER",
#                     "TOTAL POSTS", "IS REPLY", "REPLY TO", "HASHTAGS"])
# file.close()

# with open('RE_tweets.csv', 'wb') as file:
#     writer = unicodecsv.writer(file, delimiter = ',', quotechar = '"')
#     # Write header row.
#     writer.writerow(["TWEET", "USER_ID", "TWEET_ID", "TWT_POST_DT", "USER_NM", "USER_ACCT", "FOLLOWER_CNT", "FRIENDS_CNT",
#                      "LOCATION", "ACCT_CREATE_DT", "GEO_ENABLED", "USER_URL", "RETWEET_CNT", "COMPSENTIMENT", "POSSENTIMENT",
#                      "NEGSENTIMENT", "NEUSENTIMENT", "SENTIMENT_OA", "POSCNT", "NEGCNT", "BLOBPOL", "BLOBSUB", "COORDINATES", "VERIFIED USER",
#                      "TOTAL POSTS", "IS REPLY", "REPLY TO", "HASHTAGS"])
# file.close()

In [6]:
#---------------------------------------------------------------------------------------------------------------------
#           CREATE CONNECTION TO TWEEPY STREAM ESTABLISHING USER KEYS 
#           CREATE VADER SENTIMENT ANALSIS VARIABLE
#--------------------------------------------------------------------------------------------------------------------- 
# Variables that contains the user credentials to access Twitter API 
CONSUMER_KEY = 'GwXA2Yi7UlZnYqIjXWfX6QiL6'
CONSUMER_SECRET = '1sSFQN3nAFN7W4XoMeZgGkbQRbblVlEnJCwH8qLYEBImtHnvsQ'
ACCESS_TOKEN = '284793426-rSUoWDbldYXfRUPSHmTLC1eqPrtR3H7uaGt6Iahb'
ACCESS_SECRET = 'FYVXamm0BZMIeZCfr4vGJgsJ0fyGn4Ea2ESuVPHq1aiFv'  

# Setup tweepy to authenticate with Twitter credentials:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

# Use the VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

#---------------------------------------------------------------------------------------------------------------------
# Create the api to connect to twitter with your credentials
# wait_on_rate_limit= True;  will make the api to automatically wait for rate limits to replenish
# wait_on_rate_limit_notify= Ture;  will make the api  to print a notification when Tweepyis waiting for rate limits to replenish
#---------------------------------------------------------------------------------------------------------------------
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

In [7]:
#---------------------------------------------------------------------------------------------------------------------
#           CREATE FUNCTION TO REMOVE EMOJIS FROM PASSED STRING VARIABLE
#---------------------------------------------------------------------------------------------------------------------  
def give_emoji_free_text(text):
    try:
        RE_EMOJI = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
        clean_text = RE_EMOJI.sub(r'', text)
        return clean_text
    except:
        return text
        pass

In [8]:
cols = ["TWEET", "USER_ID", "TWEET_ID", "TWT_POST_DT", "USER_NM", "USER_ACCT", "FOLLOWER_CNT", "FRIENDS_CNT",
                             "LOCATION", "ACCT_CREATE_DT", "GEO_ENABLED", "USER_URL", "RETWEET_CNT", "COMPSENTIMENT", "POSSENTIMENT",
                             "NEGSENTIMENT", "NEUSENTIMENT", "SENTIMENT_OA", "POSCNT", "NEGCNT", "BLOBPOL", "BLOBSUB", "COORDINATES", "VERIFIED USER",
                             "TOTAL POSTS", "IS REPLY", "REPLY TO", "HASHTAGS"]

In [9]:
#---------------------------------------------------------------------------------------------------------------------
#           CREATE AND OPEN STREAM LISTENER AND DEFINE VARIABLES TO BE SAVED
#---------------------------------------------------------------------------------------------------------------------  
class StreamListener(tweepy.StreamListener):   

    def on_timeout(self):
        print('Snoozing Zzzzzz')
        
    def on_exception(self, exception):
       print(exception)
       return
        
    def on_error(self, status_code):
        if status_code == 420:
            return False
        else:
            print(status_code)
                                  
    def on_status(self, status):   
        self.limit = 5
        self.start_time = time.time()
        
        global freqvar
        global coiner
        
        if 'extended_tweet' in status._json:
            tweet = status.extended_tweet['full_text'].replace('&amp;','+')
        else:
            tweet = status.text.replace('&amp;','+')

#---------------------------------------------------------------------------------------------------------------------
#                             IF ANY EXCLUDED WORDS ARE IN TWEET, SKIP SAVING IT ALTOGETHER
#---------------------------------------------------------------------------------------------------------------------                
        if re.compile('|'.join(EXCLUDE),re.IGNORECASE).search(tweet): 
            pass
        else:                 
#---------------------------------------------------------------------------------------------------------------------
#                             TWEET FILE SECTION
#---------------------------------------------------------------------------------------------------------------------                         
#                             DEFINE FIELDS TO PULL FROM METADATA
#---------------------------------------------------------------------------------------------------------------------                 
            POSCNT = 0
            NEGCNT = 0

            user_nm = give_emoji_free_text(status.user.name) # name of the user, e.g. "Wei Xu"    
            location = give_emoji_free_text(status.user.location) # posted location user has listed
            twt_post_dt = status.created_at.strftime('%m/%d/%Y %H:%M') # when the tweet posted  
            tweet_id = status.id # This is the tweet's id 
            verified_user = status.user.verified 
            user_acct = status.user.screen_name # name of the user account, e.g. "cocoweixu"            
            user_id = status.user.id # This is the user id number 
            total_posts = status.user.statuses_count            
            follower_cnt = status.user.followers_count # number of followers the poster has
            friends_cnt = status.user.friends_count # number of followers the poster has            
            acct_create_dt = status.user.created_at.strftime('%m/%d/%Y %H:%M') # user profile created date time "Thu Jan 13 23:15:12 +0000 2011"
            geo_enabled = status.user.geo_enabled # are they geocode enabled posters
            user_url = status.user.url # did they post their website                
            retweet_cnt = status.retweet_count # number of retweets                
            coordinates = status.coordinates # geographic location coordinates
            is_reply = status.in_reply_to_status_id # is a reply?
            reply_to = status.in_reply_to_screen_name # screen name this post was in reply to

            hashtags3 = status.entities['hashtags']
            if hashtags3:
                hashtags = [ht['text'] for ht in hashtags3]
            else:
                hashtags = ''                    

#---------------------------------------------------------------------------------------------------------------------
#                             SENTIMENT ANALYSIS
#---------------------------------------------------------------------------------------------------------------------
            CompSentiment = analyzer.polarity_scores(tweet)['compound']
            PosSentiment = analyzer.polarity_scores(tweet)['pos'] 
            NegSentiment = analyzer.polarity_scores(tweet)['neg'] 
            NeuSentiment = analyzer.polarity_scores(tweet)['neu']   

#             print('Tweet: ', tweet)  
#             print('Hashtags4: ', hashtags, '\n')

            if CompSentiment >= 0.05: 
                sentiment_OA ="Positive"
                POSCNT = 1
            elif CompSentiment <= - 0.05: 
                sentiment_OA ="Negative"
                NEGCNT = 1
            else: 
                sentiment_OA ="Neutral"

#                 if "RT @" not in tweet:  
#                     print("Tweet was rated as ", analyzer.polarity_scores(tweet)['pos']*100, "% Positive") 
#                     print("Tweet was rated as ", analyzer.polarity_scores(tweet)['neu']*100, "% Neutral") 
#                     print("Tweet was rated as ", analyzer.polarity_scores(tweet)['neg']*100, "% Negative")        
#                     print('Overall the Tweet is ', sentiment_OA, ' in sentiment.')                
#                     print('Compound Sentiment Score: ', CompSentiment*100, "\n")

#---------------------------------------------------------------------------------------------------------------------
#                             SECONDARY SENTIMENT ANALYSIS
#---------------------------------------------------------------------------------------------------------------------               
#                 Polarity:range of [-1,1] 
#                 1 means positive statement  
#                 -1 means a negative statement

#                 Subjectivity: range of [0,1] 
#                 1 means opinion/emotion/judgement
#                 0 means factual info
            blob = TextBlob(tweet)
            sent2 = blob.sentiment
            BlobPol = sent2.polarity*100
            BlobSub = sent2.subjectivity*100

#                 if "RT @" not in tweet:  
#                     print('The Tweet is ', BlobPol, '% in Polarity.')
#                     print('The Tweet is ', BlobSub, '% in Subjectivity.')

            # NEED TO RUN SENTIMENT ANALYSIS USING EMOJIS FOR MORE ACCURATE MEASUREMENT OF SENTIMENT USING VADER ALGORITHM
            tweet = give_emoji_free_text(tweet)   

            row = [tweet, user_id, tweet_id, twt_post_dt, user_nm, user_acct, follower_cnt, friends_cnt, location, acct_create_dt, 
                   geo_enabled, user_url, retweet_cnt, CompSentiment, PosSentiment, NegSentiment, NeuSentiment, sentiment_OA, POSCNT, NEGCNT,
                   BlobPol, BlobSub, coordinates, verified_user, total_posts, is_reply, reply_to, hashtags]                  
        
# #---------------------------------------------------------------------------------------------------------------------
# #                             PULL DATA FROM CSV FOR CHARTING
# #---------------------------------------------------------------------------------------------------------------------                 
            data = pd.read_csv('tweets.csv', encoding = "ISO-8859-1") 
            df = data[['TWT_POST_DT', 'POSSENTIMENT', 'COMPSENTIMENT', 'NEGSENTIMENT', "POSCNT", "NEGCNT", "FOLLOWER_CNT", "FRIENDS_CNT"]]
            df['TWT_POST_DT'] = pd.to_datetime(df['TWT_POST_DT'])
            df.set_index('TWT_POST_DT', inplace=True)   

            if coiner <= 20: 
                #print("FREQVAR2: ", freqvar, " COUNTROWS: ", len(df.POSSENTIMENT.resample(freqvar).mean())) 
                coiner+=1
            else:
                clear_output()
                coiner=0
                #print("FREQVAR2: ", freqvar, " COUNTROWS: ", len(df.POSSENTIMENT.resample(freqvar).mean())) 

            if len(df.POSSENTIMENT.resample(freqvar).mean()) >= 20 and freqvar == '1min':     
                print("case 1")
                freqvar = '2Min'
            elif len(df.POSSENTIMENT.resample(freqvar).mean()) >= 20 and freqvar == '2Min': 
                print("case 2")
                freqvar = '5Min'    
            elif len(df.POSSENTIMENT.resample(freqvar).mean()) >= 20 and freqvar == '5Min':  
                print("case 3")
                freqvar = '10Min' 
            elif len(df.POSSENTIMENT.resample(freqvar).mean())>= 20 and freqvar == '10Min':  
                print("case 4")
                freqvar = '30Min' 
            elif len(df.POSSENTIMENT.resample(freqvar).mean()) >= 20 and freqvar == '30Min':  
                print("case 5")
                freqvar = '60Min' 
#                 else:
#                     print("case 6: Running as intended")
# #---------------------------------------------------------------------------------------------------------------------
# #                             RESAMPLE DATA INTO 5 MINUTE SEGMENTS
# #---------------------------------------------------------------------------------------------------------------------                
            Summary5min = pd.DataFrame()                

            Summary5min['POSSENTIMENT'] = df.POSSENTIMENT.resample(freqvar).mean()
            Summary5min['COMPSENTIMENT'] = df.COMPSENTIMENT.resample(freqvar).mean()
            Summary5min['NEGSENTIMENT'] = df.NEGSENTIMENT.resample(freqvar).mean()                 

            Summary5min['POSCNT'] = df.POSCNT.resample(freqvar).sum() 
            Summary5min['NEGCNT'] = df.NEGCNT.resample(freqvar).sum() 
            Summary5min['FOLLOWER_CNT'] = df.FOLLOWER_CNT.resample(freqvar).sum() 
            Summary5min['FRIENDS_CNT'] = df.FRIENDS_CNT.resample(freqvar).sum()       

            Summary5min = Summary5min.truncate(before=Summary5min.index.min(), after=Summary5min.index.max())  

#---------------------------------------------------------------------------------------------------------------------
#                             DETERMINE WHICH TWEET FILT TO SAVE TO 
#---------------------------------------------------------------------------------------------------------------------         
            if "RT @" not in tweet: 
                print(tweet)
                SAVEFILE = 'tweets.csv'
            
            elif "RT @" in tweet:
                SAVEFILE = 'RE_tweets.csv'

#---------------------------------------------------------------------------------------------------------------------
#                             SAVE DATA TO PREDETERMINED FILES
#---------------------------------------------------------------------------------------------------------------------     
            try:
                columns = ['POSSENTIMENT', 'COMPSENTIMENT', 'NEGSENTIMENT', 'POSCNT', 'NEGCNT', 'FOLLOWER_CNT', 'FRIENDS_CNT']
                Summary5min.to_csv('tweets_chart.csv', columns=columns, float_format='%11.6f', index=True, encoding = "ISO-8859-1")

            except:
#                     print("=============================================================================")
#                     print("TWEET_CHART DIDN'T SAVE.")
#                     print("============================================================================= \n")
                pass  

            try:
                with open(SAVEFILE, 'a', newline='') as csvFile:
                    writer = csv.writer(csvFile)
                    writer.writerow(row)
                csvFile.close()     
#                     print("============================================================================= \n")

            except:
#                     print("=============================================================================")
#                     print("TWEETS/RETWEETS DIDN'T SAVE.")
#                     print("============================================================================= \n")
                pass                   

#---------------------------------------------------------------------------------------------------------------------
#                             RELEASE MEMORY / CLEAR VARIABLE
#--------------------------------------------------------------------------------------------------------------------- 
            row = []   


In [10]:
#---------------------------------------------------------------------------------------------------------------------
#                             IF EXCLUSIONARY KEYWORDS TO BE USED, LIST THEM HERE
#---------------------------------------------------------------------------------------------------------------------  
EXCLUDE = ["nascar"]

In [12]:
#---------------------------------------------------------------------------------------------------------------------
#                             OPEN STREAM SEARCH
#---------------------------------------------------------------------------------------------------------------------    
print('Running... Start Time: ', start)
print("============================================================================= \n")
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
cnter = 0
freqvar = '1min'

try:
#     stream.filter(track=["xfinity","comcast","comcast cares","comcastcares","comcast_cares","comcast flex","comcast_flex", 
#                         "xfinity cares","xfinitycares","xfinity_cares","xfinity flex","xfinity_flex"], languages=["en"], stall_warnings=True)
    
    stream.filter(track=["#DemDebate","DemDebate","Cory","Booker","DemDebate2"], languages=["en"], stall_warnings=True)

except BaseException as e:
    if urllib.error.HTTPError:
        print(urllib.error.HTTPError.reason, urllib.error.HTTPError.getcode)
        pass
    elif cnter <= 5:
        print("Error on_data: %s, Pausing..." % str(e), " ", time.time()-start)
        cnter += 1
    else:
        cnter = 0
        clear_output()
        print('Running... Start Time: ', start)
        print("============================================================================= \n")
        print("Error on_data: %s, Pausing..." % str(e), " ", time.time()-start)
    time.sleep(2)   
    pass

#     except http_incompleteRead as e:
#         print("http.client Incomplete Read error: %s" % str(e))
#         print("~~~ Restarting stream search in 5 seconds 1... ~~~")
#         time.sleep(5)
#         #restart stream - simple as return true just like previous exception?
# #         return True
#         continue

#     except urllib3_incompleteRead as e:
#         print("urllib3 Incomplete Read error: %s" % str(e))
#         print("~~~ Restarting stream search in 5 seconds 2... ~~~")
#         time.sleep(5)
# #         return True        
#         continue

except (ProtocolError, AttributeError):
    print("Skipping error.")
    pass

except KeyboardInterrupt:
    print("KeyboardInterrupt has been caught.")
    pass

Latinas make 54 cents for every white man’s dollar. 
That’s it, that’s the tweet. #DemDebate2
... your move, @clorox.
If she can be a moral advisor to Bill Clinton and get Steve Tyler off heroin, she can probably bring World Peace..
Me when candidates ignore and shut down CNN’s divisive questions #DemDebate2 https://t.co/MNGPBAkewy
The exchange I saw between @TulsiGabbard + @KamalaHarris showed that Gabbard all but SPANKED Harris + sent her to bed without any dinner! She looked STUNNED.🤣 #DemDebate
Gillibrand should be attacking Ivanka trump for her bullshit women in the workplace initiative instead of Joe Biden..

#DemDebate2
Biden's response is that he wrote the Violence Against Women Act. 

#DemDebate
@rachelkaras if only these things were as important as Cory Booker's Kool-Aid
@CNN Please cory double speaks... his actions negates his kumbaya speak.
<property object at 0x000001A6B8391BD8> <function addinfourl.getcode at 0x000001A6B839CE18>
